In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import missingno as msn
import datetime
from pandas_profiling import ProfileReport
import sklearn

In [2]:
df = pd.read_parquet("escooter_history_2022.parquet")


In [3]:
df.loc[df.temp > 50, "temp"] = df.temp.div(100)

In [4]:
df_2 = df.groupby(pd.Grouper(key="datetime",freq="D")).agg(Anzahl_pro_Tag=("registered_customer","count"),temp=("temp","mean"),humidity=("humidity","mean"),windspeed=("windspeed","mean")).reset_index()

In [5]:
df_2

,datetime,Anzahl_pro_Tag,temp,humidity,windspeed
0,2020-01-04,985,16.489909,79.678173,16.226271
1,2020-01-05,801,14.415006,67.141074,15.875136
2,2020-01-06,1349,8.440104,40.287620,14.724723
3,2020-01-07,1562,9.051498,56.411012,12.351703
4,2020-01-08,1600,9.649350,38.230000,14.113182
...,...,...,...,...,...
726,2021-12-30,4231,10.475495,59.472229,22.242182
727,2021-12-31,8144,10.419155,58.653487,10.341263
728,2022-01-01,5932,10.551763,76.322151,8.237662
729,2022-01-02,5182,10.778680,46.243342,23.974396


In [7]:
bins = [0,5,10,15,20,25,30,35,40,45,50]
df_2['temp'] = pd.cut(df_2['temp'], bins)

In [ ]:
bins = [0,1000,2000,3000,4000,5000,6000,7000,8000,9000,10000,11000,12000,13000,14000,15000]
df_2['Anzahl_pro_Tag'] = pd.cut(df_2['Anzahl_pro_Tag'], bins)

In [15]:
df_slice = pd.crosstab(df_2.Anzahl_pro_Tag,df_2.temp)

In [17]:
df_slice

temp,"(0, 5]","(5, 10]","(10, 15]","(15, 20]","(20, 25]","(25, 30]","(30, 35]","(35, 40]"
Anzahl_pro_Tag,,,,,,,,
"(0, 1000]",1,7,1,4,0,0,0,0
"(1000, 2000]",1,26,19,8,0,2,0,0
"(2000, 3000]",0,10,26,20,5,6,0,0
"(3000, 4000]",0,5,38,26,15,14,3,2
"(4000, 5000]",0,0,20,31,24,33,29,1
"(5000, 6000]",0,0,25,21,25,22,12,0
"(6000, 7000]",0,1,8,10,21,21,28,1
"(7000, 8000]",0,0,2,10,24,33,16,2
"(8000, 9000]",0,0,4,6,4,11,13,0


In [16]:
from scipy.stats import chi2_contingency
#Achtung Die CHiquadrattest übergeben Tabelle muss immer absolute Häufigkeiten entahlten
#darf also keine relativen Häufigkeiten erhatlen

ch2, p, dof, expected = chi2_contingency(df_slice)
print(p)

1.107949425628107e-52


In [20]:
bins = [-1,10,20,30,40,50,60,70,80,90,100]
df_2['humidity'] = pd.cut(df_2['humidity'], bins)

In [22]:
df_slice2= pd.crosstab(df_2.Anzahl_pro_Tag,df_2.humidity)

In [23]:
df_slice2

humidity,"(-1, 10]","(10, 20]","(20, 30]","(30, 40]","(40, 50]","(50, 60]","(60, 70]","(70, 80]","(80, 90]","(90, 100]"
Anzahl_pro_Tag,,,,,,,,,,
"(0, 1000]",1,0,0,3,1,1,2,1,3,1
"(1000, 2000]",0,1,1,8,13,9,5,6,9,4
"(2000, 3000]",0,0,1,8,14,9,5,12,12,6
"(3000, 4000]",0,0,0,8,25,16,22,18,11,3
"(4000, 5000]",0,0,2,11,19,29,43,21,12,1
"(5000, 6000]",0,0,3,7,19,27,28,18,2,1
"(6000, 7000]",0,0,3,9,18,28,19,12,1,0
"(7000, 8000]",0,0,0,13,25,25,16,7,1,0
"(8000, 9000]",0,0,0,2,9,12,9,6,0,0


In [24]:
ch2, p, dof, expected = chi2_contingency(df_slice2)
print(p)

2.5320898017483612e-09


In [28]:
bins = [0,5,10,15,20,25,30,35]
df_2['windspeed'] = pd.cut(df_2['windspeed'], bins)

In [30]:
df_slice3 = pd.crosstab(df_2.Anzahl_pro_Tag,df_2.windspeed)

In [31]:
df_slice3

windspeed,"(0, 5]","(5, 10]","(10, 15]","(15, 20]","(20, 25]","(25, 30]","(30, 35]"
Anzahl_pro_Tag,,,,,,,
"(0, 1000]",0,3,1,2,5,2,0
"(1000, 2000]",3,10,15,14,7,4,2
"(2000, 3000]",2,13,22,21,6,2,1
"(3000, 4000]",2,25,30,21,15,10,0
"(4000, 5000]",5,33,55,29,11,5,0
"(5000, 6000]",7,29,30,26,10,2,1
"(6000, 7000]",1,23,34,19,12,1,0
"(7000, 8000]",4,29,35,14,4,1,0
"(8000, 9000]",1,9,17,8,2,1,0


In [32]:
ch2, p, dof, expected = chi2_contingency(df_slice3)
print(p)

0.11076692481191482
